In [ ]:
# В этом задании вам необходимо будет реализовать статистический Spell Checker

In [11]:
# !pip install razdel corus numpy nltk tqdm

# !wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [19]:
import re
import razdel
import corus
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import tqdm
from collections import Counter
import pickle

In [4]:
# Данные: Корпус русских текстов для n-gram статистики --> возьмем новостный корпус с corus
# Словарь слов русского языка (чем больше, тем лучше)
# Предложения, которые необходимо исправить

In [ ]:
# Шаг 1:

# На первом шаге коррекции наших текстов определим такие токены, которым требуется исправление.
# Для этого проведем статистическую бинарную классификацию токенов в наших предложениях
# (1- токен содержит опечатку, 0- токен не содержит опечатку)

# Определять неправильные токены будем с помощью формулы расчета "подозрительности" триграмм из статьи 1975 года 
# "Computer Detection of Typographical Errors  R. Morris, L. Cherry". Статья приложена.

# Сначала напишем формулу для получения n-gram слова. Для формулы нам нужны только биграммы и триграммы, но мы напишем
# функцию, которая возвращает n-граммы для любого заданного n.

In [25]:
def ngram(word, n):
    ngrams = []
    if len(word) < n:
        return ngrams
    elif len(word) == n:
        ngrams.append(word)
        return ngrams
    else:
        for i in range(len(word)):
            if i == (len(word) - 2):
                break
            string = ''
            while len(string) != n:
                if len(string) == n:
                    break
                else:
                    string += word[i]
                    i += 1
            ngrams.append(string)
    return ngrams

# как --> ['как']
# не --> []
# шарик --> ['шар', 'ари', 'рик']
# неправильный --> ['неп', 'епр', 'пра', 'рав', 'ави', 'вил', 'иль', 'льн', 'ьны', 'ный']


assert ngram('неправильный', 3) == [''.join(g) for g in list(nltk.ngrams('неправильный', 3))]

In [42]:
# Логика сбора статистики такова:
# 1) Идем по текстам корпуса новостей
# 2) Токенизируем тексты с помощью razdel.tokenize()
# 3) Приводим каждый токен к нижнему регистру
# 4) Токены, которые содержат только символы кириллицы, копим в статистику 
#    (делим токен на биграмы и триграммы и копим статистику в Counter)

In [26]:
# Корпус русских текстов
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)

next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [27]:
def is_cyrillic(word):
    if re.search('[а-яА-ЯЁё]', word):
        return True

ngram_stats = Counter()

for record in tqdm.tqdm(records):
    # Токенизируем текст
    tokens = word_tokenize(record.text)
    for token in tokens:
        if not is_cyrillic(token):
            continue
        # Нижний регистр
        token = token.lower()
        # Получаем триграммы и биграммы
        n_grams = ngram(token, 2) + ngram(token, 3)
        for g in n_grams:
            ngram_stats[g] += 1

with open('counter_ngrams.pickle', 'wb') as f:
    pickle.dump(ngram_stats, f)

739350it [1:26:02, 143.23it/s]


In [28]:
ngram_stat.most_common()

NameError: name 'ngram_stat' is not defined

In [114]:
import pandas as pd

df = pd.read_csv('broken_texts.csv.gz', compression='gzip')[['text']]

In [183]:
df['text'].iloc[11]

'в 1996 г . плоучил звание заслуженныйй профессор харьковского государственного университета .'

Формула для расчета подозрительности триграмы (обозначается xyz) выглядит следующим образом:

$$ i(T) = \frac{1}{2}[log(xy) + log(yz)] - log(xyz) $$

Если биграма или триграма отсутствует в словаре, то значение логарифма по задумке авторов сразу равно -10

Эту логику лучше вынести в отдельную функцию

In [ ]:
with open('counter_ngrams.pickle', 'rb') as f:
    ngram_stats = pickle.load(f)

In [184]:
def count_log(g):
    if g in ngram_stats:
        pass

In [ ]:
# Соберем всё вместе

    def pecularity(trigram):
    return 

def get_scores(token):
#    В конечном итоге скоры для одного слова должны выглядеть как-то так:
#     {'плоучил': 
#          {'пло': 2.59,
#           'лоу': 3.29,
#           'оуч': 4.09,
#           'учи': 1.56,
#           'чил': 2.40}
#     }
    return
# Если токен имеет триграмы с скорами > 4, то мы считаем, что такой токен имеет ошибку.
# То есть его частота в нашем корпусе частот практически незначительна
    



In [ ]:
# Токены, в которых есть значения выше 4: пробуем восстановить

# По аналогии с решением предыдущей задачи воспользуйтесь n-gram преобразованием, чтобы найти top-k ближайших кандидатов
# для исправления токена с помощью списка слов русского языка и функции scipy.cdist 

# Будем использовать и униграмы, и биграмы, и триграмы для этой части задания


In [186]:
# Слова русского языка
words = list(pd.read_csv('russian_words.zip', compression='zip').values.flatten())

print(words[120:130])

len(words)

['абаками', 'Абакан', 'Абакана', 'Абакане', 'абаканец', 'Абаканом', 'абаканская', 'абаканские', 'абаканский', 'абаканским']


1532629

In [ ]:
def predict_candidates(word, k):
    # TODO: predict top k most similar words from russian word dictionary
    return candidates